<a href="https://colab.research.google.com/github/xbarusui/study_gpt2-japanese_V2/blob/main/study_gpt2_japaneseV2_AuthorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> 作家AI の文章学習・生成</h1>
メニュー

*   1.前準備(googleドライブ)
*   2.前準備(モジュールインストール)
*   3.作家AI学習
*   4.作家AIの文章生成



# 1.前準備(googleドライブ)


In [ ]:
#@title googleドライブに接続して gpt2work というフォルダを作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/gpt2work/'
%cd '/content/drive/My Drive/gpt2work/'

<h1> 上記の説明 </h1>

1．下のセルを実行すると、URLと入力欄が表示されます。

![Googleドライブ認証](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage1.png?raw=true)

2．青いURLをクリックすると、新しいタブが開いて、colabと連携するアカウントを聞かれます（聞かれなかった場合、次に進んで下さい）

![アカウントの選択](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage2.png?raw=true)

3．「このアプリをGoogleからダウンロードしたことをご確認ください」と出ますので確認の上、「ログイン」を選択してください。

![代替テキスト](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage3.png?raw=true)

4．許可すると、認証コードの横にコピーボタンが表示されますので、コピーボタンをクリックして下さい。

![認証コード](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage4.png?raw=true)

5．colabの画面に戻ってきて、URLの下の入力欄にカーソルを合わせて、「Ctrl+V」で貼り付けて下さい。文字列は・で表示されます。

![認証コード貼り付け](https://github.com/xbarusui/study_gpt2-japanese_V2/blob/main/authimage5.png?raw=true)




# 2.前準備(モジュールインストール)

In [ ]:
#@title 学習済みモデルと周辺モジュールインストール
!git clone https://github.com/rinnakk/japanese-pretrained-models
%cd japanese-pretrained-models
!pip install -r requirements.txt

# 再度ディレクトリ移動
%cd '/content/drive/My Drive/gpt2work/'

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

# 初めての場合は、gpt2work以下に下のフォルダがないか確認し、無ければモジュールをgithubダウンロード
import os
path = 'study_gpt2-japanese_V2'
is_dir = os.path.isdir(path)

if is_dir is False:
  !git clone https://github.com/xbarusui/study_gpt2-japanese_V2


# 作家AI学習（学習不要なら省略可能）

In [ ]:
#@title 初期位置に移動

%cd '/content/drive/My Drive/gpt2work/'

In [ ]:
#@title 学習させたいテキストファイルをUPLOADする

from google.colab import files
uploaded = files.upload()

In [ ]:
#@title パラメータ設定後に学習


#@markdown 



#@markdown ### 作家AIのモデルをドロップダウンから選択してください

#@markdown Default以外の番号を選択すると、そこに学習した作家AIが格納されます

authormodel = "1" #@param ["Default", "1","2","3","4","5","6","7","8","9","10"]

#@markdown 

if authormodel == "Default" :
  savefolder = "output/"
else:
  savefolder = authormodel + "/"

#@markdown ### UPLOADしたファイル名を入力してください
upload_file = "junhonnovel.txt"  #@param {type:"string"}

#@markdown

#@markdown ### num_train_epochs（何エポック学習するかを設定します）
num_train_epochs = 10  #@param {type:"number"}


# ファインチューニングの実行
!python study_gpt2-japanese_V2/gpt2japanese_study.py \
    --model_name_or_path=rinna/japanese-gpt2-xsmall \
    --train_file={upload_file} \
    --validation_file={upload_file} \
    --do_train \
    --do_eval \
    --num_train_epochs={num_train_epochs} \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir={savefolder} \
    --use_fast_tokenizer=False \
    --overwrite_output_dir

# 作家AIの文章生成

In [ ]:
#@title パラメータ設定後に文章生成

#@markdown 

#@markdown ### 作家AIのモデルをドロップダウンから選択してください

#@markdown Default だと初期の学習済みモデルを利用し、数字を選択すると上で学習した番号に格納されたモデルを使用します

authormodel = "Default" #@param ["Default", "1","2","3","4","5","6","7","8","9","10"]

#@markdown 

from transformers import T5Tokenizer, AutoModelForCausalLM

if authormodel == "Default" :
  # ライブラリをインポートします。
  # 学習済みモデルをダウンロードします。
#  tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-xsmall")
#  tokenizer.do_lower_case = True 
  savefolder = "rinna/japanese-gpt2-xsmall"
else:
  savefolder = authormodel + "/"


#@markdown ### 書かせる文章の最初の部分を記載してください

noveltext = ""  #@param {type:"string"}

#@markdown ### 文字数を選択してください

novellength = 128  #@param {type:"number"}

#@markdown ### 何回生成するかを選択してください

novelseq =   4#@param {type:"number"}


# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-xsmall")
model = AutoModelForCausalLM.from_pretrained(savefolder)

# 推論
input = tokenizer.encode( noveltext , return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=novellength, num_return_sequences=novelseq)

#print(tokenizer.batch_decode(output))

# 出力されたコードを文章に戻します。
DecodedOutput = tokenizer.batch_decode(output)

# 生成された文章を表示します。
print('#'*40)
print(f'冒頭が「{noveltext}」となる文章を出力します。')
print('#'*40)

for j in range(novelseq):
  print('#'*40)
  print(f'### 文章案{j} ####')
  print('#'*40)
  i = 0
  while i < len(DecodedOutput[j]):
    print(DecodedOutput[j].replace('</s>','')[i:i+80]) 
    i = i+80 